In [10]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [11]:
L, W = 0.01, 0.005  # Length and width of the domain (in m)
lambda_ = 5.0e9   # Elastic constant (Pa)
mu = 5.0e9         # Shear modulus (Pa)
h = 0.01          # Thickness (m)

In [12]:
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 2)
        )

    def forward(self, x, y):
        inputs = torch.cat([x, y], dim=1)
        return self.net(inputs)

In [13]:
def strain_tensor(u_x, u_y, x, y):
    u_x_x = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), retain_graph=True, create_graph=True)[0]
    u_y_y = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u_y), retain_graph=True, create_graph=True, allow_unused=True)[0]
    u_x_y = torch.autograd.grad(u_x, y, grad_outputs=torch.ones_like(u_x), retain_graph=True, create_graph=True)[0]
    u_y_x = torch.autograd.grad(u_y, x, grad_outputs=torch.ones_like(u_y), retain_graph=True, create_graph=True)[0]

    E_xx = u_x_x
    E_yy = u_y_y
    E_xy = 0.5 * (u_x_y + u_y_x)
    return E_xx, E_yy, E_xy

In [14]:
def stress_tensor(E_xx, E_yy, E_xy):
    scale_factor = 1e9  # Scaling factor for material constants
    trace_E = E_xx + E_yy
    sigma_xx = h * ((lambda_ / scale_factor) * trace_E + 2 * (mu / scale_factor) * E_xx)
    sigma_yy = h * ((lambda_ / scale_factor) * trace_E + 2 * (mu / scale_factor) * E_yy)
    sigma_xy = h * (2 * (mu / scale_factor) * E_xy)
    return sigma_xx, sigma_yy, sigma_xy

In [15]:
def physics_loss(model, x, y):
    x.requires_grad_(True)
    y.requires_grad_(True)
    u = model(x, y)
    u_x, u_y = u[:, 0:1], u[:, 1:2]

    E_xx, E_yy, E_xy = strain_tensor(u_x, u_y, x, y)
    sigma_xx, sigma_yy, sigma_xy = stress_tensor(E_xx, E_yy, E_xy)

    sigma_xx_x = torch.autograd.grad(sigma_xx, x, grad_outputs=torch.ones_like(sigma_xx), retain_graph=True, create_graph=True)[0]
    sigma_xy_y = torch.autograd.grad(sigma_xy, y, grad_outputs=torch.ones_like(sigma_xy), retain_graph=True, create_graph=True)[0]
    sigma_yy_y = torch.autograd.grad(sigma_yy, y, grad_outputs=torch.ones_like(sigma_yy), retain_graph=True, create_graph=True)[0]
    sigma_xy_x = torch.autograd.grad(sigma_xy, x, grad_outputs=torch.ones_like(sigma_xy), retain_graph=True, create_graph=True)[0]

    residual_x = sigma_xx_x + sigma_xy_y
    residual_y = sigma_yy_y + sigma_xy_x

    loss_equilibrium = torch.mean(residual_x**2 + residual_y**2)
    return loss_equilibrium

In [16]:
def boundary_condition_loss(model, L, W):
    # Boundary A: u_x = 0, u_y = 0 at x = -L/2
    y_A = torch.linspace(-W / 2, W / 2, 100).reshape(-1, 1).requires_grad_()
    u_A = model(-L / 2 * torch.ones_like(y_A, requires_grad=True), y_A)
    loss_A = torch.mean(u_A**2)

    # Boundary D: u_x = 0.025 * L, u_y = 0 at x = L/2
    y_D = torch.linspace(-W / 2, W / 2, 100).reshape(-1, 1).requires_grad_()
    u_D = model(L / 2 * torch.ones_like(y_D, requires_grad=True), y_D)
    loss_D = torch.mean((u_D[:, 1:2]**2) + (u_D[:, 0:1] - 0.025 * L)**2)

    # Boundary B: traction-free (σ_xx = σ_xy = 0) at y = W/2
    x_B = torch.linspace(-L / 2, L / 2, 100).reshape(-1, 1).requires_grad_()
    y_B = W / 2 * torch.ones_like(x_B, requires_grad=True)
    u_B = model(x_B, y_B)
    u_B_x, u_B_y = u_B[:, 0:1], u_B[:, 1:2]
    E_xx_B, E_yy_B, E_xy_B = strain_tensor(u_B_x, u_B_y, x_B, y_B)
    sigma_xx_B, sigma_yy_B, sigma_xy_B = stress_tensor(E_xx_B, E_yy_B, E_xy_B)
    loss_B = torch.mean(sigma_yy_B**2 +sigma_xy_B**2)

    # Boundary C: traction-free (σ_xx = σ_xy = 0) at y = -W/2
    x_C = torch.linspace(-L / 2, L / 2, 100).reshape(-1, 1).requires_grad_()
    y_C = -W / 2 * torch.ones_like(x_C, requires_grad=True)
    u_C = model(x_C, y_C)
    u_C_x, u_C_y = u_C[:, 0:1], u_C[:, 1:2]
    E_xx_C, E_yy_C, E_xy_C = strain_tensor(u_C_x, u_C_y, x_C, y_C)
    sigma_xx_C, sigma_yy_C, sigma_xy_C = stress_tensor(E_xx_C, E_yy_C, E_xy_C)
    loss_C = torch.mean(sigma_yy_C**2 + sigma_xy_C**2)

    return loss_A + loss_D + loss_B / 1e18  + loss_C / 1e18 # later divide B and C by 10**18

In [17]:
def train_pinn_two_phases(model, optimizer,
                          n_epochs_phase1, n_epochs_phase2,
                          n_points, L, W,
                          pde_weight_phase2=0.01):
    """
    Phase I: PDE emphasis (ignore or lightly weight BC).
    Phase II: BC emphasis + small PDE penalty (pde_weight_phase2).
    """
    pde_loss_hist = []
    bc_loss_hist = []

    # -- Phase I: PDE Focus --
    for epoch in range(n_epochs_phase1):
        # Sample interior points
        x = torch.rand((n_points, 1)) * L - L/2
        y = torch.rand((n_points, 1)) * W - W/2

        loss_pde = physics_loss(model, x, y)
        loss_total = loss_pde  # BC not included or very tiny weight

        optimizer.zero_grad()
        loss_total.backward()
        optimizer.step()

        pde_loss_hist.append(loss_pde.item())
        if epoch % 500 == 0:
            print(f"[Phase I, Epoch {epoch}] PDE Loss: {loss_pde.item():.3e}")

    # -- Phase II: BC Focus (plus small PDE penalty) --
    for epoch in range(n_epochs_phase2):
        x = torch.rand((n_points, 1)) * L - L/2
        y = torch.rand((n_points, 1)) * W - W/2

        loss_bc = boundary_condition_loss(model, L, W)
        loss_pde = physics_loss(model, x, y)

        loss_total = loss_bc + pde_weight_phase2 * loss_pde

        optimizer.zero_grad()
        loss_total.backward()
        optimizer.step()

        bc_loss_hist.append(loss_bc.item())
        if epoch % 500 == 0:
            print(f"[Phase II, Epoch {epoch}] BC Loss: {loss_bc.item():.3e}, PDE Loss: {loss_pde.item():.3e}")

    return pde_loss_hist, bc_loss_hist

def train_pinn_combined(model, optimizer, n_epochs, n_points, L, W, alpha=1.0, beta=1.0):
    """
    Single-phase training that combines PDE (alpha * PDE) and BC (beta * BC).
    """
    history = []
    for epoch in range(n_epochs):
        x = torch.rand((n_points, 1)) * L - L/2
        y = torch.rand((n_points, 1)) * W - W/2

        loss_pde = physics_loss(model, x, y)
        loss_bc = boundary_condition_loss(model, L, W)
        loss_total = alpha * loss_pde + beta * loss_bc

        optimizer.zero_grad()
        loss_total.backward()
        optimizer.step()

        history.append(loss_total.item())
        if epoch % 500 == 0:
            print(f"[Phase III, Epoch {epoch}] PDE Loss: {loss_pde.item():.3e}, BC Loss: {loss_bc.item():.3e}, Total: {loss_total.item():.3e}")
    return history

In [18]:
 # Phase settings
    n_points = 1000
    n_epochs_phase1 = 3000
    n_epochs_phase2 = 3000

    # 1) Phase I & II: Two-Stage Training
    print("=== Starting Phase I & II ===")
    pde_loss_hist, bc_loss_hist = train_pinn_two_phases(
        model, optimizer,
        n_epochs_phase1, n_epochs_phase2,
        n_points, L, W,
        pde_weight_phase2=0.01  # how strongly PDE is enforced in Phase II
    )

    # 2) Phase III: Combined Refinement (optional)
    print("\n=== Starting Phase III (Combined PDE+BC) ===")
    n_epochs_phase3 = 2000
    alpha = 1.0
    beta = 1.0
    combined_history = train_pinn_combined(
        model, optimizer, n_epochs_phase3, n_points, L, W,
        alpha=alpha, beta=beta
    )

IndentationError: unexpected indent (497779928.py, line 2)

In [ ]:
# Initialize model and optimizer
torch.set_default_dtype(torch.float64)
model = PINN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model in two phases
n_points = 1000
n_epochs_phase1 = 5000
n_epochs_phase2 = 5000
pde_loss_hist, bc_loss_hist = train_pinn_two_phases(
    model, optimizer, n_epochs_phase1, n_epochs_phase2, n_points, L, W
)


In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error 

In [ ]:
file_path = '/Users/murat/Downloads/data.csv'  # Update this with the correct path
comparison_data = pd.read_csv(file_path)

In [ ]:
x_values = torch.tensor(comparison_data['X'].values, dtype=torch.float32).reshape(-1, 1)
y_values = torch.tensor(comparison_data['Y'].values, dtype=torch.float32).reshape(-1, 1)
u_x_actual = comparison_data['u_x_actual'].values
u_y_actual = comparison_data['u_y_actual'].values

In [ ]:
model.eval()
with torch.no_grad():
    u_pred = model(x_values, y_values) 
    u_x_pred = u_pred[:, 0].reshape(-1).numpy()
    u_y_pred = u_pred[:, 1].reshape(-1).numpy()

In [ ]:
comparison_data['u_x_pred'] = u_x_pred
comparison_data['u_y_pred'] = u_y_pred

In [ ]:
comparison_data['error_u_x'] = abs(comparison_data['u_x_actual'] - comparison_data['u_x_pred'])
comparison_data['error_u_y'] = abs(comparison_data['u_y_actual'] - comparison_data['u_y_pred'])
comparison_data['percent_error_u_x'] = abs(comparison_data['u_x_actual'] - comparison_data['u_x_pred']) / abs(comparison_data['u_x_actual'])
comparison_data['percent_error_u_y'] = abs(comparison_data['u_y_actual'] - comparison_data['u_y_pred']) / abs(comparison_data['u_y_actual'])


In [ ]:
print(comparison_data)

In [ ]:
comparison_data.to_csv('/Users/murat/Downloads/comparison_results.csv', index=False)